In [16]:
from importlib import reload

import pandas as pd

import bookcrossing_dataset

In [54]:
reload(bookcrossing_dataset)
bookcrossing_dataset.process_dataset()

Reading items...


In [55]:
items_df = bookcrossing_dataset.items_df()
with pd.option_context('display.max_colwidth', 1000):
    display(items_df)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,title,description,image_slug,image_size
item_id,,,,,,,,,,,,
0195153448,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg,Classical Mythology,None,[0195153448],31578
0002005018,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg,Clara Callan,None,[0002005018],52460
0060973129,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg,Decision in Normandy,None,[0060973129],0
0374157065,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,None,[0374157065],66859
0393045218,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg,The Mummies of Urumchi,None,[0393045218],26440
...,...,...,...,...,...,...,...,...,...,...,...,...
0440400988,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0440400988.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0440400988.01.LZZZZZZZ.jpg,There's a Bat in Bunk Five,None,[0440400988],0
0525447644,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0525447644.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0525447644.01.LZZZZZZZ.jpg,From One to One Hundred,None,[0525447644],95476
006008667X,006008667X,Lily Dale : The True Story of the Town that Talks to the Dead,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/006008667X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/006008667X.01.LZZZZZZZ.jpg,Lily Dale : The True Story of the Town that Talks to the Dead,None,[006008667X],47019


In [56]:
reviews_df = bookcrossing_dataset.reviews_df()
reviews_df

,user_id,item_id,rating
0,276725,034545104X,0.0
1,276726,0155061224,2.5
2,276727,0446520802,0.0
3,276729,052165615X,1.5
4,276729,0521795028,3.0
...,...,...,...
1149775,276704,1563526298,4.5
1149776,276706,0679447156,0.0
1149777,276709,0515107662,5.0
1149778,276721,0590442449,5.0


In [58]:
MIN_IMAGE_SIZE = 1000

def only_items_with_image(items_df):
    item_has_image = items_df.loc[items_df['image_size'] >= MIN_IMAGE_SIZE].index
    return items_df.loc[item_has_image]


def only_with_enough_reviews(reviews_df):
    item_id_sizes = reviews_df.groupby('item_id').size()
    items_with_enough_reviews = item_id_sizes.loc[item_id_sizes >= 5].index

    # Filter out items with less than 5 reviews or not in items_df
    reviews_df = reviews_df.loc[
        reviews_df['item_id'].isin(items_with_enough_reviews)
    ]
    
    user_id_sizes = reviews_df.groupby('item_id').size()
    users_with_enough_reviews = user_id_sizes.loc[user_id_sizes >= 5].index
    reviews_df = reviews_df.loc[
        reviews_df['item_id'].isin(users_with_enough_reviews)
    ]

    return reviews_df

def filter_items_and_reviews(items_df, reviews_df):
    # Filter out items that do not have an image
    items_df = only_items_with_image(items_df)

    # Do not consider reviews for items that are not in the items_df
    print('Filtering out reviews without image...')
    reviews_df = reviews_df.loc[
        reviews_df['item_id'].isin(items_df.index)
    ]

    # Filter out items with less than 5 reviews or not in items_df
    item_id_sizes = reviews_df.groupby('item_id').size()
    items_with_enough_reviews = item_id_sizes.loc[item_id_sizes >= 5].index

    has_enough_reviews = reviews_df['item_id'].isin(items_with_enough_reviews)

    filtered_reviews = reviews_df.loc[item_has_image & has_enough_reviews]

    # Filter out users with less than 5 reviews
    user_id_sizes = filtered_reviews.groupby('item_id').size()
    users_with_enough_reviews = user_id_sizes.loc[user_id_sizes >= 5].index
    filtered_reviews = filtered_reviews.loc[
        filtered_reviews['user_id'].isin(users_with_enough_reviews)
    ]

    # Do not consider items that are not in the reviews_df
    print('Filtering out items without reviews...')
    filtered_items = items_df.filter(
        filtered_reviews['item_id'].unique(), axis='index'
    )

filter_items_and_reviews(items_df, reviews_df)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,title,description,image_slug,image_size
item_id,,,,,,,,,,,,
0195153448,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Classical Mythology,None,[0195153448],31578
0002005018,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara Callan,None,[0002005018],52460
0060973129,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,Decision in Normandy,None,[0060973129],0
0374157065,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...,None,[0374157065],66859
0393045218,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,The Mummies of Urumchi,None,[0393045218],26440
...,...,...,...,...,...,...,...,...,...,...,...,...
0440400988,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,There's a Bat in Bunk Five,None,[0440400988],0
0525447644,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,From One to One Hundred,None,[0525447644],95476
006008667X,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,Lily Dale : The True Story of the Town that Ta...,None,[006008667X],47019


In [14]:
100 * len(reviews_df) /  (reviews_df['item_id'].nunique() * reviews_df['user_id'].nunique())

0.005665114569655576